In [2]:
import glob
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [3]:
base_path = "/media/tensor/EXTDRIVE/projects/virtual-try-on/dataset/lip_mpv_dataset/"

In [4]:
dense_path = base_path + "lip_dense/"

In [5]:
image_path = base_path + "lip_images/"

In [6]:
dest_path = base_path + "lip_textures/"

In [7]:
dense_list = glob.glob(dense_path + "*")

In [9]:
for dense in tqdm(dense_list):
#     try:
        name = dense.split("/")[-1]
        dense = dense_path + name
#         model_image = image_path + name.split(".")[0] + ".jpg"
        model_image = image_path + name[:-4] + ".jpg"
        dense_array = np.load(dense)
        model_image_pil = Image.open(model_image)
        model_image_array = np.asarray(model_image_pil)[:dense_array.shape[1],:dense_array.shape[2],:]

        solution = 75
        solution_float = float(solution) - 1 

        U = dense_array[1, :, :]
        V = dense_array[2, :, :]
        parts = list()
        for PartInd in range(1, 25):
            actual_part = np.zeros((solution, solution, 3))
            x, y = np.where(dense_array[0,:,:] == PartInd)
            if len(x) == 0:
                parts.append(actual_part)
                continue
            u_current_points = U[x, y]
            v_current_points = V[x, y]


            tex_map_coords = ((255-v_current_points)*solution_float/255.).astype(int), (u_current_points*solution_float/255.).astype(int)
            for c in range(3):
                actual_part[tex_map_coords[0], tex_map_coords[1], c] = model_image_array[x, y, c]

            parts.append(actual_part)
        TextureIm  = np.zeros([solution*6,solution*4,3]);
        for i in range(4):
            for j in range(6):
                TextureIm[ (solution*j):(solution*j+solution)  , (solution*i):(solution*i+solution) ,: ] = parts[i*6+j]
        TextureIm = TextureIm.transpose(1, 0, 2)


        

        normal_pil = Image.fromarray(np.uint8(TextureIm))
#         normal_pil.save(dest_path + name.split(".")[0] + ".jpg")
        normal_pil.save(dest_path + name[:-4] + ".jpg")
#     except:
#         print(name)